In [13]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/burke/Documents/research/scrooge")

oneDocscreeningServices = pd.read_excel("SingleProvider10YearLifetimeScreeningServices.xlsx")
oneDocscreeningServices['year'] = oneDocscreeningServices.visitDate.dt.year

oneDocPanel = pd.read_excel("SingleProvider10YearPanel.xlsx")
oneDocVisits = pd.read_excel("SingleProvider10YearLifetimeVisits.xlsx")
oneDocVisits['year'] = oneDocVisits.visitDate.dt.year

lastYearScreening = oneDocscreeningServices[oneDocscreeningServices.year==2026]
learYearVisits = oneDocVisits[oneDocVisits.year==2026]

In [17]:
oneDocVisits.year.value_counts()

2018    4140
2019    3430
2020    3149
2021    3028
2022    2904
2024    2837
2023    2829
2026    2640
2025    2626
2027    2540
Name: year, dtype: int64

In [29]:
oneDocPanel.iloc[-1]

patientID                                87200
gender                                  Female
age                                         28
raceEth                     Non-Hispanic White
WTINT2YR                               68967.6
WTMEC2YR                               69107.4
sdmvpsu                                      2
sdmvstra                                   121
bmi                                       20.3
selfReportDiabetes                          No
smokingStatus                                0
selfReportHtn                               No
selfReportHyperlipidemia                   Yes
probWeight                         0.000290474
Name: 60613, dtype: object

In [27]:
import datetime
import numpy.random as random
startOfYearTime = datetime.datetime.strptime("1/1/2018" , "%m/%d/%Y")
endOfYearTIme = datetime.datetime.strptime("12/31/2018" , "%m/%d/%Y")

timesForVisits = sorted([startOfYearTime + datetime.timedelta(days=(random.rand() * (endOfYearTIme-startOfYearTime).total_seconds() / 86400)) for i in range(0,100)  ], reverse=True)




In [28]:
timesForVisits

[datetime.datetime(2018, 12, 28, 21, 7, 5, 634003),
 datetime.datetime(2018, 12, 27, 10, 0, 38, 826918),
 datetime.datetime(2018, 12, 27, 3, 23, 58, 586421),
 datetime.datetime(2018, 12, 25, 13, 19, 26, 267524),
 datetime.datetime(2018, 12, 22, 22, 34, 27, 1723),
 datetime.datetime(2018, 12, 21, 14, 49, 53, 588199),
 datetime.datetime(2018, 12, 18, 13, 22, 15, 787457),
 datetime.datetime(2018, 12, 11, 8, 22, 8, 498946),
 datetime.datetime(2018, 12, 6, 3, 0, 9, 699914),
 datetime.datetime(2018, 12, 4, 2, 20, 38, 586712),
 datetime.datetime(2018, 12, 1, 3, 30, 15, 87607),
 datetime.datetime(2018, 11, 27, 8, 44, 25, 989028),
 datetime.datetime(2018, 11, 22, 23, 17, 31, 194442),
 datetime.datetime(2018, 11, 20, 15, 40, 13, 278447),
 datetime.datetime(2018, 11, 14, 6, 59, 49, 648637),
 datetime.datetime(2018, 11, 10, 6, 47, 56, 829450),
 datetime.datetime(2018, 11, 10, 4, 7, 28, 238419),
 datetime.datetime(2018, 11, 2, 17, 54, 39, 774541),
 datetime.datetime(2018, 11, 1, 3, 25, 25, 765741),